<a href="https://colab.research.google.com/github/VaibhavShintre/Introduction-to-Data-Science/blob/main/CSC_442_Project_Group_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Names: Lavan Aditya, Amrish Naranappa, Vaibhav Shintre, Austin Bressler

In [1]:
# install the neccesary libraries
from google.colab import files
import io
import pandas as pd
import numpy as np

# upload the data file
uploaded = files.upload()

Saving student-por.csv to student-por.csv
Saving student-mat.csv to student-mat.csv


In [2]:
# read in the database
StudentMath = pd.read_csv('student-mat.csv', delimiter=';')
StudentPort = pd.read_csv('student-por.csv', delimiter=';')

# checking if the files were read correctly
print(StudentMath)
print(StudentPort)

    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... famrel freetime  goout  Dalc  Walc health 

In [83]:
# renaming the column variable names to differentiate the G1 , G2 and G3 values from the math and portugal files
mathG3Only = StudentMath.drop(['G1','G2'], axis=1)
portG3Only = StudentPort.drop(['G1','G2'], axis=1)

Sm_temp = mathG3Only.rename(columns={'G3': 'MG3', "paid": "paidM", "failures": "failuresM", "absences": "absencesM"}, inplace=False)
print(Sm_temp)
sp_temp = portG3Only.rename(columns={'G3': 'PG3', "paid": "paidP", "failures": "failuresP", "absences": "absencesP"}, inplace=False)
print(sp_temp)


    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... internet romantic  famrel  freetime  goout

In [84]:
# merging on the columns that are supposed to be matching according to the original source.
sm_inter = pd.merge(Sm_temp, sp_temp, 'inner')

# printing the result to verify that there is a correct amount of matching files.
print(sm_inter.info())
# print(sm_inter["guardian_x"].equals(sm_inter["guardian_y"]))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 35 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      370 non-null    object
 1   sex         370 non-null    object
 2   age         370 non-null    int64 
 3   address     370 non-null    object
 4   famsize     370 non-null    object
 5   Pstatus     370 non-null    object
 6   Medu        370 non-null    int64 
 7   Fedu        370 non-null    int64 
 8   Mjob        370 non-null    object
 9   Fjob        370 non-null    object
 10  reason      370 non-null    object
 11  guardian    370 non-null    object
 12  traveltime  370 non-null    int64 
 13  studytime   370 non-null    int64 
 14  failuresM   370 non-null    int64 
 15  schoolsup   370 non-null    object
 16  famsup      370 non-null    object
 17  paidM       370 non-null    object
 18  activities  370 non-null    object
 19  nursery     370 non-null    object
 20  higher    